In [15]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
import urllib.request
import os

In [10]:
URL = 'https://comic.naver.com/webtoon/weekday'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

title = soup.find_all('a', {'class' : 'title'}) # a태그에서 class='title'인 html소스를 찾아 할당
id_list = []
title_num_list = []
link_list = []
title_list = []
author_list = []
day_list = []
genre_list = []
story_list = []
imgSrc_list = []
num = 0

driver = webdriver.Chrome('C:/Users/Administrator/Desktop/JavaSpring/Webtoon_recommender/Webtoon_recommender/python_webtoon_crawler/chromedriver.exe')
driver.get(URL)

for i in range(len(title)):
    sleep(0.5)

    page = driver.find_elements_by_class_name('title')
    page[i].click() #월요일 첫 번째 웹툰부터 순서대로 클릭

    sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 이동한 페이지 주소 파싱

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1]

    t = title[i].text
    if (t in title_list):  # 한 작품이 요일 두 개 이상이면 요일만 추가
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue
    
    link = soup.find_all('div', attrs = {'class' : 'thumb'})
    link = link[0].find('a').attrs['href']
    
    title_num = link.replace('/webtoon/list?titleId=','')
    #print(title_num)
    
    image_src = soup.find_all('div', attrs = {'class' : 'thumb'})
    image_src = image_src[0].find('img').attrs['src']
    
    id_list.append(num) ; num += 1
    title_list.append(t)
    day_list.append(day)
    imgSrc_list.append(image_src)
    link_list.append(link)
    title_num_list.append(title_num)
    
    author = soup.find_all('h2')
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] # 7칸의 공백 후 8번 째부터 작가 이름
    author_list.append(author)

    genre = soup.find('span', {'class' : 'genre'}).text
    genre_list.append(genre)

    story = soup.find_all('p')
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n').replace('&lt;', '<').replace('&gt;', '>') # <br>을 개행으로
    story_list.append(story)
    #print(image_src)
    
    driver.back() # 뒤로 가기

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23880/341244883.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Administrator/Desktop/JavaSpring/Webtoon_recommender/Webtoon_recommender/python_webtoon_crawler/chromedriver.exe')
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_23880/341244883.py:23: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  page = driver.find_elements_by_class_name('title')


758037
https://shared-comic.pstatic.net/thumb/webtoon/758037/thumbnail/thumbnail_IMAG06_794bcc1e-23aa-4c35-a335-b5d21b4bc2ab.jpg
648419
https://shared-comic.pstatic.net/thumb/webtoon/648419/thumbnail/thumbnail_IMAG06_44119122-8568-4293-9159-a99e2a6b0558.jpg
783052
https://shared-comic.pstatic.net/thumb/webtoon/783052/thumbnail/thumbnail_IMAG06_f8e8c275-a327-48a9-9573-de64dadb9d60.jpg
728750
https://shared-comic.pstatic.net/thumb/webtoon/728750/thumbnail/thumbnail_IMAG06_50aeb870-ee3e-47e4-bb38-a0d01e49ec8f.jpg
602910
https://shared-comic.pstatic.net/thumb/webtoon/602910/thumbnail/thumbnail_IMAG06_032a19d0-fe41-452e-854c-1030b4147f7e.jpg
654774
https://shared-comic.pstatic.net/thumb/webtoon/654774/thumbnail/thumbnail_IMAG06_8ec78896-0b7f-44c3-9eb6-2ba9ab3b07f0.jpg
774863
https://shared-comic.pstatic.net/thumb/webtoon/774863/thumbnail/thumbnail_IMAG06_56ecb055-dc10-43e6-bfa2-4a07fd1c7f30.jpg
795297
https://shared-comic.pstatic.net/thumb/webtoon/795297/thumbnail/thumbnail_IMAG06_9d37dbef-

794156
https://shared-comic.pstatic.net/thumb/webtoon/794156/thumbnail/thumbnail_IMAG06_155ee6e7-a30e-441c-bdc5-bf6653c31366.jpg
774870
https://shared-comic.pstatic.net/thumb/webtoon/774870/thumbnail/thumbnail_IMAG06_38b55265-2706-4e98-9cc0-1aad0839c0fa.jpg
776256
https://shared-comic.pstatic.net/thumb/webtoon/776256/thumbnail/thumbnail_IMAG06_4e6725c8-b1b7-4b27-9e93-e01134ff9a42.jpg
769801
https://shared-comic.pstatic.net/thumb/webtoon/769801/thumbnail/thumbnail_IMAG06_05345da8-5158-4bfb-b532-e21538f7cc74.jpg
794942
https://shared-comic.pstatic.net/thumb/webtoon/794942/thumbnail/thumbnail_IMAG06_45ce8550-5fca-409f-9f45-d2da34a183d1.jpg
796368
https://shared-comic.pstatic.net/thumb/webtoon/796368/thumbnail/thumbnail_IMAG06_234d5dfa-e5d2-4ee3-84dc-1cc3969ebc7a.jpg
793662
https://shared-comic.pstatic.net/thumb/webtoon/793662/thumbnail/thumbnail_IMAG06_e54fb886-6941-4bc4-a503-490c7cdfa532.jpg
785259
https://shared-comic.pstatic.net/thumb/webtoon/785259/thumbnail/thumbnail_IMAG06_82dce408-

789029
https://shared-comic.pstatic.net/thumb/webtoon/789029/thumbnail/thumbnail_IMAG06_0b751285-9d4c-4d1f-8d14-9fd8c65123ec.jpg
761601
https://shared-comic.pstatic.net/thumb/webtoon/761601/thumbnail/thumbnail_IMAG06_fcf9f838-4449-4cd3-a413-389fb631eb79.jpg
764622
https://shared-comic.pstatic.net/thumb/webtoon/764622/thumbnail/thumbnail_IMAG06_b4d28d1c-cae3-491e-bbe7-1fe5d9697a6c.jpg
798100
https://shared-comic.pstatic.net/thumb/webtoon/798100/thumbnail/thumbnail_IMAG06_5af3abc0-ebd1-406f-af06-762eb2503b8d.jpg
790776
https://shared-comic.pstatic.net/thumb/webtoon/790776/thumbnail/thumbnail_IMAG06_ab4f31dc-1000-4f72-8ed2-7e23ac9919f2.jpg
773793
https://shared-comic.pstatic.net/thumb/webtoon/773793/thumbnail/thumbnail_IMAG06_2e5b820e-053e-4317-8947-0991993227c3.jpg
789682
https://shared-comic.pstatic.net/thumb/webtoon/789682/thumbnail/thumbnail_IMAG06_61bc4ef5-fb32-4dba-aa54-e93b896d9ca9.jpg
794105
https://shared-comic.pstatic.net/thumb/webtoon/794105/thumbnail/thumbnail_IMAG06_91913d75-

735979
https://shared-comic.pstatic.net/thumb/webtoon/735979/thumbnail/thumbnail_IMAG06_7d71d748-7db8-4b27-84bf-6333ba109911.jpg
801446
https://shared-comic.pstatic.net/thumb/webtoon/801446/thumbnail/thumbnail_IMAG06_fb8aad25-c9f3-415b-8101-e5e6dcf8d1b1.jpg
783861
https://shared-comic.pstatic.net/thumb/webtoon/783861/thumbnail/thumbnail_IMAG06_21bce92e-7070-4cc6-acd5-d3918306ed9a.jpg
796466
https://shared-comic.pstatic.net/thumb/webtoon/796466/thumbnail/thumbnail_IMAG06_646923da-e9a8-46a9-a895-be2dd9a33e34.jpg
795330
https://shared-comic.pstatic.net/thumb/webtoon/795330/thumbnail/thumbnail_IMAG06_68b51c25-bc36-427e-9bcf-5bad5d1353c0.jpg
750558
https://shared-comic.pstatic.net/thumb/webtoon/750558/thumbnail/thumbnail_IMAG06_ff74b387-a953-4356-a4a0-c45abb88384c.jpg
788167
https://shared-comic.pstatic.net/thumb/webtoon/788167/thumbnail/thumbnail_IMAG06_d9f4e7e7-3931-45c4-a655-55c21a24d8d5.jpg
765158
https://shared-comic.pstatic.net/thumb/webtoon/765158/thumbnail/thumbnail_IMAG06_76bef3ba-

741825
https://shared-comic.pstatic.net/thumb/webtoon/741825/thumbnail/thumbnail_IMAG06_eb982633-2a5d-44fd-b80c-b0536b860488.jpg
701700
https://shared-comic.pstatic.net/thumb/webtoon/701700/thumbnail/thumbnail_IMAG06_ea073683-a9d5-4f47-9125-90a7375a1f41.jpg
570506
https://shared-comic.pstatic.net/thumb/webtoon/570506/thumbnail/title_thumbnail_20130710192300_t125x101.jpg
785727
https://shared-comic.pstatic.net/thumb/webtoon/785727/thumbnail/thumbnail_IMAG06_d3fbd591-d2ac-4a76-aab1-5d293f3537ab.jpg
775631
https://shared-comic.pstatic.net/thumb/webtoon/775631/thumbnail/thumbnail_IMAG06_87c61f44-a7a5-4cee-b120-379374b7e871.jpg
797731
https://shared-comic.pstatic.net/thumb/webtoon/797731/thumbnail/thumbnail_IMAG06_68dd2411-d492-41a2-84eb-ff5841c26e0d.jpg
798331
https://shared-comic.pstatic.net/thumb/webtoon/798331/thumbnail/thumbnail_IMAG06_5524204d-695b-41ef-bf0d-8dbde8fd55ef.jpg
758659
https://shared-comic.pstatic.net/thumb/webtoon/758659/thumbnail/thumbnail_IMAG06_07ad95b1-6a6e-45bb-8f8d

796492
https://shared-comic.pstatic.net/thumb/webtoon/796492/thumbnail/thumbnail_IMAG06_da35760b-a066-4b37-b2d3-79aacb9fb1d0.jpg
641253
https://shared-comic.pstatic.net/thumb/webtoon/641253/thumbnail/thumbnail_IMAG06_7c62962d-f52d-4757-b640-4d2abcc05b29.jpg
776601
https://shared-comic.pstatic.net/thumb/webtoon/776601/thumbnail/thumbnail_IMAG06_58f9fcb8-5a99-417a-86c6-8b0ace616167.jpg
773797
https://shared-comic.pstatic.net/thumb/webtoon/773797/thumbnail/thumbnail_IMAG06_db977436-48b9-4abb-aa97-9cda3fb92a2b.jpg
318995
https://shared-comic.pstatic.net/thumb/webtoon/318995/thumbnail/title_thumbnail_20110407182655_t125x101.jpg
790245
https://shared-comic.pstatic.net/thumb/webtoon/790245/thumbnail/thumbnail_IMAG06_bf35e77a-ab38-46ab-9a61-eeec847bcae1.jpg
777767
https://shared-comic.pstatic.net/thumb/webtoon/777767/thumbnail/thumbnail_IMAG06_43be643d-73fc-49a0-a06d-7c184340fc7a.jpg
753478
https://shared-comic.pstatic.net/thumb/webtoon/753478/thumbnail/thumbnail_IMAG06_875fed07-1422-4f84-9ac4

793410
https://shared-comic.pstatic.net/thumb/webtoon/793410/thumbnail/thumbnail_IMAG06_369a1b69-c615-4827-9dd2-c20360d84785.jpg
783552
https://shared-comic.pstatic.net/thumb/webtoon/783552/thumbnail/thumbnail_IMAG06_1b176f3b-b66f-4476-be0a-8b5a75a7d56b.jpg
799478
https://shared-comic.pstatic.net/thumb/webtoon/799478/thumbnail/thumbnail_IMAG06_c0c66fe6-be54-49b6-876f-c5da0ff6d254.jpg
729036
https://shared-comic.pstatic.net/thumb/webtoon/729036/thumbnail/thumbnail_IMAG06_81e82b6a-acb5-4ea1-8eda-d6bbbad7a4b7.jpg
785855
https://shared-comic.pstatic.net/thumb/webtoon/785855/thumbnail/thumbnail_IMAG06_1e6bfba1-791f-432e-8eb1-17489cd0dca6.jpg
799226
https://shared-comic.pstatic.net/thumb/webtoon/799226/thumbnail/thumbnail_IMAG06_35d42dfb-23f2-4693-b581-b4dacced4c00.jpg
773475
https://shared-comic.pstatic.net/thumb/webtoon/773475/thumbnail/thumbnail_IMAG06_0db8297f-8369-42b4-bf3b-c1714038b5df.jpg
791847
https://shared-comic.pstatic.net/thumb/webtoon/791847/thumbnail/thumbnail_IMAG06_6bcab10a-

746833
https://shared-comic.pstatic.net/thumb/webtoon/746833/thumbnail/thumbnail_IMAG06_a86aa198-d03f-4295-af81-7d64472a3c37.jpg
797932
https://shared-comic.pstatic.net/thumb/webtoon/797932/thumbnail/thumbnail_IMAG06_d042d2c7-69af-475c-aeb1-c99d5584bc35.jpg
785260
https://shared-comic.pstatic.net/thumb/webtoon/785260/thumbnail/thumbnail_IMAG06_2ec9790a-e8be-420f-808a-692df468be13.jpg
786978
https://shared-comic.pstatic.net/thumb/webtoon/786978/thumbnail/thumbnail_IMAG06_865e3c01-c6b8-4817-af9a-36437bfb3ca7.jpg
791936
https://shared-comic.pstatic.net/thumb/webtoon/791936/thumbnail/thumbnail_IMAG06_4b0b0722-f5e9-46eb-8872-d62d8a467136.jpg
733277
https://shared-comic.pstatic.net/thumb/webtoon/733277/thumbnail/thumbnail_IMAG06_3a12884b-b94b-4b7a-bfe5-03a456b0da4f.jpg
769193
https://shared-comic.pstatic.net/thumb/webtoon/769193/thumbnail/thumbnail_IMAG06_d27b11ce-4d04-4a79-85ee-664dacf230e4.jpg
778579
https://shared-comic.pstatic.net/thumb/webtoon/778579/thumbnail/thumbnail_IMAG06_c116fea5-

796251
https://shared-comic.pstatic.net/thumb/webtoon/796251/thumbnail/thumbnail_IMAG06_469726fc-05b5-4301-9c5c-eb94b658164d.jpg
797155
https://shared-comic.pstatic.net/thumb/webtoon/797155/thumbnail/thumbnail_IMAG06_e99ed5b6-5159-4164-930d-1785182696fa.jpg
753842
https://shared-comic.pstatic.net/thumb/webtoon/753842/thumbnail/thumbnail_IMAG06_4a2c8474-15d8-4330-9e54-87fe0dd69469.jpg
790416
https://shared-comic.pstatic.net/thumb/webtoon/790416/thumbnail/thumbnail_IMAG06_18a6aa1e-3d22-43a1-8301-342fd60262c5.jpeg
786973
https://shared-comic.pstatic.net/thumb/webtoon/786973/thumbnail/thumbnail_IMAG06_17e948ef-dddf-47e3-89b3-6a443eef71e6.jpg
773473
https://shared-comic.pstatic.net/thumb/webtoon/773473/thumbnail/thumbnail_IMAG06_d93185a4-eb33-4625-bcee-b07bdfc65296.jpg
783865
https://shared-comic.pstatic.net/thumb/webtoon/783865/thumbnail/thumbnail_IMAG06_5b08f5bb-f0ab-4f2a-ae70-00e69687c058.jpg
798172
https://shared-comic.pstatic.net/thumb/webtoon/798172/thumbnail/thumbnail_IMAG06_d74ebbbb

In [14]:
total_data = pd.DataFrame()
total_data['id'] = id_list
total_data['title_num'] = title_num_list
total_data['title'] = title_list
total_data['author'] = author_list
total_data['day'] = day_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['link'] = link_list
total_data['img_src'] = imgSrc_list
print(total_data)
total_data.to_csv('naver_webtoon_data.csv', encoding = 'utf-8-sig')

      id title_id        title          author day       genre  \
0      0   758037          참교육       채용택 / 한가람   월     스토리, 액션   
1      1   648419      뷰티풀 군바리        설이 / 윤성원   월    스토리, 드라마   
2      2   783052      퀘스트지상주의        박태준 만화회사   월    스토리, 드라마   
3      3   728750    장씨세가 호위무사       김인호 / 조형근   월  스토리, 무협/사극   
4      4   602910       윈드브레이커             조용석   월    스토리, 스포츠   
..   ...      ...          ...             ...  ..         ...   
535  535   758676      조선여우스캔들  문종호 / 정후 / 차소희   일    스토리, 로맨스   
536  536   773524      거래하실래요?        99C / 백도   일    스토리, 로맨스   
537  537   795041     아마도, 굿모닝   은나 / 솜베 / 우지혜   일    스토리, 로맨스   
538  538   776143  독신마법사 기숙아파트        재윤 / 기르답   일    스토리, 로맨스   
539  539   785812     구해줘, 호구!              기천   일    스토리, 로맨스   

                                                 story  \
0    무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n<부활남>...   
1                       '여자도 군대에 간다면?'본격 여자도 군대 가는 만화!   
2    [외모지상주의], [싸움독학], [인생존망]과 세계

In [22]:
#이미지 링크로 이미지파일 저장(name = title_num)
basic_path = "C:/Users/Administrator/Desktop/JavaSpring/Webtoon_recommender/Webtoon_recommender/src/main/resources/static/img"
    
def save_file(title_num):
    for i in range(len(title_num_list)):
        f = open(basic_path + '/' + str(title_num[i]) + ".jpg", 'wb')
        img_file = urllib.request.urlopen(imgSrc_list[i])
        f.write(img_file.read())
        f.close

In [23]:
save_file(title_num_list)